In [ ]:
import torch
from transformers import PerceiverForMultimodalAutoencoding

In [ ]:
model = PerceiverForMultimodalAutoencoding.from_pretrained("deepmind/multimodal-perceiver")

In [ ]:
y = outputs.hidden_states

In [ ]:
regression_head = torch.nn.Sequential(
    torch.nn.Linear(model.config.d_latents, model.config.d_latents * 2), 
    torch.nn.GELU(), 
    torch.nn.Linear(model.config.d_latents * 2, model.config.d_latents)
    )

In [ ]:
# https://www.baeldung.com/cs/instance-vs-batch-normalization


with torch.no_grad():
    y = y[-5:]  # take the last k transformer layers
    # Follow the same layer normalization procedure for text and vision
    y = [torch.layer_norm(tl.float(), tl.shape[-1:]) for tl in y]
    y = sum(y) / len(y)
    if True: # noralize targets
        y = torch.layer_norm(y.float(), y.shape[-1:])

    # # # Use instance normalization for audio
    # y = [torch.nn.functional.instance_norm(tl.float()) for tl in y]
    # y = sum(y) / len(y)
    # if True: # normalize targets
    #     y = torch.nn.functional.instance_norm(y.transpose(1, 2)).transpose(1, 2)

print(outputs.hidden_states[-1:][0].size())
x = regression_head(outputs.hidden_states[-1:][0])
print(x.size())
print(y.size())

In [ ]:
l1 = torch.nn.SmoothL1Loss()
mse = torch.nn.MSELoss()

In [ ]:
l1(x, y)

In [ ]:
mse(x, y)